# 1. Import và kiểm tra bộ dữ liệu

## 1.1 Import dữ liệu

In [126]:
import pandas as pd

# Đọc dữ liệu từ file Excel
df = pd.read_excel("Group12_Data4Apriori_FP-Growth.xlsx", sheet_name="Online Retail")

# Hiển thị thông tin tổng quan về DataFrame
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None


## 1.2 Xem bộ dữ liệu

### Xem số dòng dữ liệu bị thiếu

In [127]:
# In ra tổng số dòng dữ liệu
print("Tổng số dòng dữ liệu:", len(df))

# In ra số dòng thiếu dữ liệu
print("Số dòng thiếu dữ liệu:")
print(df.isnull().sum())

Tổng số dòng dữ liệu: 541909
Số dòng thiếu dữ liệu:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


### Xem tổng số hóa đơn và hóa đơn bị hủy

In [128]:
# Nhóm dữ liệu theo InvoiceNo, lấy danh sách sản phẩm trong mỗi hóa đơn
transactions = df.groupby('InvoiceNo')['Description'].apply(list)

# Tổng số hóa đơn
total_invoices = transactions.shape[0]
print(f"Tổng số hóa đơn: {total_invoices}")

# Số hóa đơn bị hủy (InvoiceNo bắt đầu bằng 'C')
canceled_invoices = df[df['InvoiceNo'].astype(str).str.startswith('C')].groupby('InvoiceNo').ngroups
print(f"Số hóa đơn bị hủy: {canceled_invoices}")

Tổng số hóa đơn: 25900
Số hóa đơn bị hủy: 3836


### Xem số lượng hóa đơn của các nước

In [129]:
# Đếm số lượng hóa đơn theo từng quốc gia
invoice_count_by_country = df.groupby('Country')['InvoiceNo'].nunique()

# In ra tổng số lượng hóa đơn của từng quốc gia
print(invoice_count_by_country)

# In tổng số hóa đơn
print("Tổng số hóa đơn:", df['InvoiceNo'].nunique())

Country
Australia                  69
Austria                    19
Bahrain                     4
Belgium                   119
Brazil                      1
Canada                      6
Channel Islands            33
Cyprus                     20
Czech Republic              5
Denmark                    21
EIRE                      360
European Community          5
Finland                    48
France                    461
Germany                   603
Greece                      6
Hong Kong                  15
Iceland                     7
Israel                      9
Italy                      55
Japan                      28
Lebanon                     1
Lithuania                   4
Malta                      10
Netherlands               101
Norway                     40
Poland                     24
Portugal                   71
RSA                         1
Saudi Arabia                2
Singapore                  10
Spain                     105
Sweden                     46
Sw

# 2. Tiền xử lý dữ liệu

In [130]:
# Loại bỏ hóa đơn chỉ còn 1 sản phẩm
transactions = [t for t in transactions if len(t) > 1]

# Loại bỏ các dòng có Quantity và UnitPrice không hợp lệ (<= 0)
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Xoá dòng chứa giá trị thiếu ở cột 'Description'
df.dropna(subset=['Description'], inplace=True)

# Xoá hóa đơn huỷ (bắt đầu bằng 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Lọc 3000 giao dịch (InvoiceNo)
invoice_list = df['InvoiceNo'].unique()[:3000]
df = df[df['InvoiceNo'].isin(invoice_list)]

# Nhóm dữ liệu theo InvoiceNo, lấy danh sách sản phẩm trong mỗi giao dịch
transactions = df.groupby('InvoiceNo')['Description'].apply(list).tolist()


# In ra 5 giao dịch đầu tiên
for t in transactions[:5]:
    print(t)

print("Số lượng giao dịch:", len(transactions))


['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER', 'KNITTED UNION FLAG HOT WATER BOTTLE', 'RED WOOLLY HOTTIE WHITE HEART.', 'SET 7 BABUSHKA NESTING BOXES', 'GLASS STAR FROSTED T-LIGHT HOLDER']
['HAND WARMER UNION JACK', 'HAND WARMER RED POLKA DOT']
['ASSORTED COLOUR BIRD ORNAMENT', "POPPY'S PLAYHOUSE BEDROOM ", "POPPY'S PLAYHOUSE KITCHEN", 'FELTCRAFT PRINCESS CHARLOTTE DOLL', 'IVORY KNITTED MUG COSY ', 'BOX OF 6 ASSORTED COLOUR TEASPOONS', 'BOX OF VINTAGE JIGSAW BLOCKS ', 'BOX OF VINTAGE ALPHABET BLOCKS', 'HOME BUILDING BLOCK WORD', 'LOVE BUILDING BLOCK WORD', 'RECIPE BOX WITH METAL HEART', 'DOORMAT NEW ENGLAND']
['JAM MAKING SET WITH JARS', 'RED COAT RACK PARIS FASHION', 'YELLOW COAT RACK PARIS FASHION', 'BLUE COAT RACK PARIS FASHION']
['BATH BUILDING BLOCK WORD']
Số lượng giao dịch: 3000


# 3. Chuyển đổi dữ liệu

In [131]:
# Chuyển tất cả phần tử trong các giao dịch về chuỗi
transactions = [[str(item) for item in transaction] for transaction in transactions]

# Mã hóa dữ liệu giao dịch để sử dụng trong thuật toán Apriori và FP-Growth
from mlxtend.preprocessing import TransactionEncoder

# Mã hóa dữ liệu giao dịch
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)

df_encoded = pd.DataFrame(te_array, columns=te.columns_)
df_encoded.head()

,4 PURPLE FLOCK DINNER CANDLES,OVAL WALL MIRROR DIAMANTE,SET 2 TEA TOWELS I LOVE LONDON,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 IVORY ROSE PEG PLACE SETTINGS,...,ZINC FINISH 15CM PLANTER POTS,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE 2 WALL PLANTER,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE CHARGER SMALL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART LATTICE TRAY OVAL,ZINC METAL HEART DECORATION,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# 4. Chọn ngưỡng support

###
Support là tỉ lệ số giao dịch chứa nhóm sản phẩm đó trên tổng số giao dịch

- Nếu support cao quá, chỉ ra được các nhóm rất phổ biến

- Nếu support thấp quá, ra nhiều nhóm không ý nghĩa, thuật toán chạy lâu

Cách chọn ngưỡng Support:
- Xem tổng số giao dịch
- Chọn số lần muốn xuất hiện tối thiểu mong muốn, ở bài toán này là chọn 100 lần

In [132]:
min_support = 100 / len(transactions) 
min_support = round(min_support, 3)  # làm tròn 3 số thập phân
print("Ngưỡng support nên chọn:", min_support)


Ngưỡng support nên chọn: 0.033


# 5. Chọn ngưỡng confidence

###
1. Hiểu về confidence

- Confidence là xác suất mặt hàng B được mua khi đã mua mặt hàng A

- VD: Confidence = 0.4 nghĩa là cứ 100 giao dịch mua A thì có 40 giao dịch mua cả B

2. Confidence được chọn trong bài toán: 0.5, vì:
- Nếu chọn quá thấp thì kết quả ra nhiều, gây tốn dung lượng bộ nhớ và khó phân tích
- Nếu chọn quá cao thì có thể bỏ lỡ nhiều mẫu tiềm năng


# 6. Thực hiện thuật toán Apriori và FP-Growth

In [133]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
import time

min_confidence = 0.5

# Apriori
start = time.time()
frequent_itemsets_apriori = apriori(df_encoded, min_support=min_support, use_colnames=True)
rules_apriori = association_rules(frequent_itemsets_apriori, metric="confidence", min_threshold=min_confidence)
apriori_time = time.time() - start

# FP-Growth
start = time.time()
frequent_itemsets_fpgrowth = fpgrowth(df_encoded, min_support=min_support, use_colnames=True)
rules_fpgrowth = association_rules(frequent_itemsets_fpgrowth, metric="confidence", min_threshold=min_confidence)
fpgrowth_time = time.time() - start


# 7. Hiển thị kết quả, Nhận xét và so sánh

In [134]:
print(f"Số luật Apriori: {len(rules_apriori)}, thời gian: {apriori_time:.2f}s")
print(f"Số luật FP-Growth: {len(rules_fpgrowth)}, thời gian: {fpgrowth_time:.2f}s")

print("Luật Apriori:")
display(rules_apriori.sort_values('confidence', ascending=False))

print("Luật FP-Growth:")
display(rules_fpgrowth.sort_values('confidence', ascending=False))

Số luật Apriori: 12, thời gian: 0.32s
Số luật FP-Growth: 12, thời gian: 0.55s
Luật Apriori:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
2,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.049333,0.049667,0.035333,0.716216,14.420461,1.0,0.032883,3.348794,0.978949,0.554974,0.701385,0.713813
3,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.049667,0.049333,0.035333,0.711409,14.420461,1.0,0.032883,3.294171,0.979292,0.554974,0.696433,0.713813
1,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.054000,0.056667,0.036667,0.679012,11.982571,1.0,0.033607,2.938846,0.968864,0.495495,0.659730,0.663036
9,(SET OF 6 SPICE TINS PANTRY DESIGN),(SET OF 3 CAKE TINS PANTRY DESIGN ),0.058000,0.086000,0.039000,0.672414,7.818765,1.0,0.034012,2.790105,0.925799,0.371429,0.641591,0.562951
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.056667,0.054000,0.036667,0.647059,11.982571,1.0,0.033607,2.680333,0.971603,0.495495,0.626912,0.663036
4,(HEART OF WICKER LARGE),(HEART OF WICKER SMALL),0.084000,0.108667,0.054333,0.646825,5.952381,1.0,0.045205,2.523775,0.908297,0.392771,0.603768,0.573413
8,(RED HANGING HEART T-LIGHT HOLDER),(WHITE HANGING HEART T-LIGHT HOLDER),0.069000,0.153333,0.044000,0.637681,4.158790,1.0,0.033420,2.336800,0.815838,0.246729,0.572064,0.462319
10,(WOODEN PICTURE FRAME WHITE FINISH),(WOODEN FRAME ANTIQUE WHITE ),0.054000,0.063333,0.034333,0.635802,10.038986,1.0,0.030913,2.571864,0.951785,0.413655,0.611177,0.588954
7,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.053000,0.096667,0.033333,0.628931,6.506181,1.0,0.028210,2.434407,0.893664,0.286533,0.589222,0.486879
11,(WOODEN FRAME ANTIQUE WHITE ),(WOODEN PICTURE FRAME WHITE FINISH),0.063333,0.054000,0.034333,0.542105,10.038986,1.0,0.030913,2.065977,0.961269,0.413655,0.515968,0.588954


Luật FP-Growth:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
9,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.049333,0.049667,0.035333,0.716216,14.420461,1.0,0.032883,3.348794,0.978949,0.554974,0.701385,0.713813
10,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.049667,0.049333,0.035333,0.711409,14.420461,1.0,0.032883,3.294171,0.979292,0.554974,0.696433,0.713813
2,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.054000,0.056667,0.036667,0.679012,11.982571,1.0,0.033607,2.938846,0.968864,0.495495,0.659730,0.663036
11,(SET OF 6 SPICE TINS PANTRY DESIGN),(SET OF 3 CAKE TINS PANTRY DESIGN ),0.058000,0.086000,0.039000,0.672414,7.818765,1.0,0.034012,2.790105,0.925799,0.371429,0.641591,0.562951
1,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.056667,0.054000,0.036667,0.647059,11.982571,1.0,0.033607,2.680333,0.971603,0.495495,0.626912,0.663036
7,(HEART OF WICKER LARGE),(HEART OF WICKER SMALL),0.084000,0.108667,0.054333,0.646825,5.952381,1.0,0.045205,2.523775,0.908297,0.392771,0.603768,0.573413
5,(RED HANGING HEART T-LIGHT HOLDER),(WHITE HANGING HEART T-LIGHT HOLDER),0.069000,0.153333,0.044000,0.637681,4.158790,1.0,0.033420,2.336800,0.815838,0.246729,0.572064,0.462319
3,(WOODEN PICTURE FRAME WHITE FINISH),(WOODEN FRAME ANTIQUE WHITE ),0.054000,0.063333,0.034333,0.635802,10.038986,1.0,0.030913,2.571864,0.951785,0.413655,0.611177,0.588954
6,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.053000,0.096667,0.033333,0.628931,6.506181,1.0,0.028210,2.434407,0.893664,0.286533,0.589222,0.486879
4,(WOODEN FRAME ANTIQUE WHITE ),(WOODEN PICTURE FRAME WHITE FINISH),0.063333,0.054000,0.034333,0.542105,10.038986,1.0,0.030913,2.065977,0.961269,0.413655,0.515968,0.588954


###
Nhận xét:

FP-Growth chạy chậm hơn Apriori

Số lượng luật và nội dung luật tương tự nhau



# 8. Các Combo Bán chéo tiềm năng (Cross-sell Combo)

In [135]:
combo_rules = rules_fpgrowth[(rules_fpgrowth['confidence'] >= 0.5) & (rules_fpgrowth['lift'] > 1.2)]

print("Các combo bán chéo tiềm năng (Cross-sell Combo):")
for idx, row in combo_rules.iterrows():
    antecedents = ', '.join(list(row['antecedents']))
    consequents = ', '.join(list(row['consequents']))
    print(f"Combo: [{antecedents}] => [{consequents}] | Support: {row['support']:.3f} | Confidence: {row['confidence']:.2f} | Lift: {row['lift']:.2f}")

# Top 5 combo mạnh nhất:
print("\nTop 5 combo bán chéo mạnh nhất:")
top_combos = combo_rules.sort_values(['confidence', 'lift'], ascending=False).head(5)
for idx, row in top_combos.iterrows():
    antecedents = ', '.join(list(row['antecedents']))
    consequents = ', '.join(list(row['consequents']))
    print(f"Combo: [{antecedents}] => [{consequents}] | Support: {row['support']:.3f} | Confidence: {row['confidence']:.2f} | Lift: {row['lift']:.2f}")

Các combo bán chéo tiềm năng (Cross-sell Combo):
Combo: [JAM MAKING SET WITH JARS] => [JAM MAKING SET PRINTED] | Support: 0.041 | Confidence: 0.54 | Lift: 6.11
Combo: [ALARM CLOCK BAKELIKE GREEN] => [ALARM CLOCK BAKELIKE RED ] | Support: 0.037 | Confidence: 0.65 | Lift: 11.98
Combo: [ALARM CLOCK BAKELIKE RED ] => [ALARM CLOCK BAKELIKE GREEN] | Support: 0.037 | Confidence: 0.68 | Lift: 11.98
Combo: [WOODEN PICTURE FRAME WHITE FINISH] => [WOODEN FRAME ANTIQUE WHITE ] | Support: 0.034 | Confidence: 0.64 | Lift: 10.04
Combo: [WOODEN FRAME ANTIQUE WHITE ] => [WOODEN PICTURE FRAME WHITE FINISH] | Support: 0.034 | Confidence: 0.54 | Lift: 10.04
Combo: [RED HANGING HEART T-LIGHT HOLDER] => [WHITE HANGING HEART T-LIGHT HOLDER] | Support: 0.044 | Confidence: 0.64 | Lift: 4.16
Combo: [JUMBO BAG PINK POLKADOT] => [JUMBO BAG RED RETROSPOT] | Support: 0.033 | Confidence: 0.63 | Lift: 6.51
Combo: [HEART OF WICKER LARGE] => [HEART OF WICKER SMALL] | Support: 0.054 | Confidence: 0.65 | Lift: 5.95
Combo